In [60]:
import pandas as pd
import re


# Data Upload

In [61]:
df = pd.read_csv('Resume1.csv')

In [62]:
df

,ID,Resume_str,Resume_html,Category
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR
...,...,...,...,...
2479,99416532,RANK: SGT/E-5 NON- COMMISSIONED OFFIC...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION
2480,24589765,"GOVERNMENT RELATIONS, COMMUNICATIONS ...","<div class=""fontsize fontface vmargins hmargin...",AVIATION
2481,31605080,GEEK SQUAD AGENT Professional...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION
2482,21190805,PROGRAM DIRECTOR / OFFICE MANAGER ...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION


In [63]:
df.shape

(2484, 4)

In [64]:
df.isnull().sum()

ID             0
Resume_str     0
Resume_html    0
Category       0
dtype: int64

In [65]:
df['Category'].unique()

array(['HR', 'DESIGNER', 'INFORMATION-TECHNOLOGY', 'TEACHER', 'ADVOCATE',
       'BUSINESS-DEVELOPMENT', 'HEALTHCARE', 'FITNESS', 'AGRICULTURE',
       'BPO', 'SALES', 'CONSULTANT', 'DIGITAL-MEDIA', 'AUTOMOBILE',
       'CHEF', 'FINANCE', 'APPAREL', 'ENGINEERING', 'ACCOUNTANT',
       'CONSTRUCTION', 'PUBLIC-RELATIONS', 'BANKING', 'ARTS', 'AVIATION'],
      dtype=object)

In [66]:
df['Category'].value_counts()

INFORMATION-TECHNOLOGY    120
BUSINESS-DEVELOPMENT      120
FINANCE                   118
ADVOCATE                  118
ACCOUNTANT                118
ENGINEERING               118
CHEF                      118
AVIATION                  117
FITNESS                   117
SALES                     116
BANKING                   115
HEALTHCARE                115
CONSULTANT                115
CONSTRUCTION              112
PUBLIC-RELATIONS          111
HR                        110
DESIGNER                  107
ARTS                      103
TEACHER                   102
APPAREL                    97
DIGITAL-MEDIA              96
AGRICULTURE                63
AUTOMOBILE                 36
BPO                        22
Name: Category, dtype: int64

In [67]:
df['Resume_str'][0]

"         HR ADMINISTRATOR/MARKETING ASSOCIATE\n\nHR ADMINISTRATOR       Summary     Dedicated Customer Service Manager with 15+ years of experience in Hospitality and Customer Service Management.   Respected builder and leader of customer-focused teams; strives to instill a shared, enthusiastic commitment to customer service.         Highlights         Focused on customer satisfaction  Team management  Marketing savvy  Conflict resolution techniques     Training and development  Skilled multi-tasker  Client relations specialist           Accomplishments      Missouri DOT Supervisor Training Certification  Certified by IHG in Customer Loyalty and Marketing by Segment   Hilton Worldwide General Manager Training Certification  Accomplished Trainer for cross server hospitality systems such as    Hilton OnQ  ,   Micros    Opera PMS   , Fidelio    OPERA    Reservation System (ORS) ,   Holidex    Completed courses and seminars in customer service, sales strategies, inventory control, loss pr

In [68]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\surya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Data Preprocessing                              

In [70]:
def NewResume(txt):
    # Remove url 
    cleanText = re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , txt)
    # Removing special characters
    cleanText = re.sub('[^a-z A-z 0-9-]+', '', cleanText)
    # remove punctuations
    cleanText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', cleanText)
    # Remove any additional spaces    
    cleanText = re.sub('\s+', ' ', cleanText)
    return cleanText

In [71]:
df['Resume_str'] = df['Resume_str'].apply(lambda x: NewResume(x))


In [72]:
df['Resume_str'][0]

' HR ADMINISTRATORMARKETING ASSOCIATEHR ADMINISTRATOR Summary Dedicated Customer Service Manager with 15 years of experience in Hospitality and Customer Service Management Respected builder and leader of customer focused teams strives to instill a shared enthusiastic commitment to customer service Highlights Focused on customer satisfaction Team management Marketing savvy Conflict resolution techniques Training and development Skilled multi tasker Client relations specialist Accomplishments Missouri DOT Supervisor Training Certification Certified by IHG in Customer Loyalty and Marketing by Segment Hilton Worldwide General Manager Training Certification Accomplished Trainer for cross server hospitality systems such as Hilton OnQ Micros Opera PMS Fidelio OPERA Reservation System ORS Holidex Completed courses and seminars in customer service sales strategies inventory control loss prevention safety time management leadership and performance assessment Experience HR AdministratorMarketing 

# change into Categorical 

In [73]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [74]:
le.fit(df['Category'])
df['Category'] = le.transform(df['Category'])

In [75]:
df.Category.unique()

array([19, 13, 20, 23,  1,  9, 18, 17,  2,  8, 22, 12, 14,  5, 10, 16,  3,
       15,  0, 11, 21,  7,  4,  6])

In [76]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

tfidf.fit(df['Resume_str'])
requredTaxt  = tfidf.transform(df['Resume_str'])

# Train & Test Split

In [77]:
from sklearn.model_selection import train_test_split

In [78]:
X_train, X_test, y_train, y_test = train_test_split(requredTaxt, df['Category'], test_size=0.2, random_state=42)

In [79]:
X_train.shape

(1987, 56647)

In [80]:
X_test.shape

(497, 56647)

# Accuracy Score

In [81]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score

clf = OneVsRestClassifier(KNeighborsClassifier())
clf.fit(X_train,y_train)
ypred = clf.predict(X_test)
print(accuracy_score(y_test,ypred))

0.4647887323943662


In [82]:
ypred

array([12, 14, 11, 10, 12, 18, 13,  9, 23, 13, 15, 18, 22,  8, 22, 18, 16,
       12, 19, 12, 13, 19, 13,  9, 14,  9,  3, 18, 20, 18,  5, 12,  0,  9,
       12, 20, 18, 10, 15, 17, 12, 20, 13, 20, 19,  0, 23, 12, 10,  9, 10,
       12, 10, 18, 17,  9,  9, 22,  3, 21,  9, 10, 12, 12, 18,  9, 19, 23,
       18, 11, 15, 22, 21, 22,  6,  6, 20, 16, 18, 20,  9, 12, 10,  6,  9,
        6, 18, 23, 21, 11, 15,  0, 20, 19, 22, 18,  0, 23,  0, 16, 18,  3,
       23, 14, 20, 22, 14, 12, 12, 12, 15, 23, 18,  9, 21, 16, 14, 11, 14,
       11, 14, 18, 18, 12, 18, 15,  0, 13, 18,  9, 18,  1, 22, 11,  9,  9,
       10,  9, 22, 18, 18, 18, 11,  9, 20, 21, 21, 23, 20, 19,  9, 20, 18,
        9,  9,  6, 20, 21, 12, 23, 20,  9, 11, 12, 21, 22,  1, 22, 12,  3,
        0, 12, 14, 12, 11, 21,  6, 22, 19, 15,  1, 18, 22, 20, 21, 20, 13,
       12, 17, 15, 22, 18, 13, 10, 16, 17, 12, 12, 14, 20, 18,  9,  6, 15,
       11,  0, 20, 20, 16, 15, 22, 23, 20,  0, 12, 16, 12,  6, 11,  0, 10,
        1,  9, 18, 10, 20

In [83]:
import pickle
with open(r"C:\Users\surya\Desktop\guvi\NLP\tfidf.pkl", 'wb') as file:
    pickle.dump(tfidf,file)
with open(r"C:\Users\surya\Desktop\guvi\NLP\clf.pkl", 'wb') as file:
    pickle.dump(clf,file)


In [84]:
myresume = df['Resume_str'][3]

In [85]:
myresume

' HR SPECIALIST Summary Dedicated Driven and Dynamic with over 20 years of customer service expertise Motivated to maintain customer satisfaction and contribute to company success with an emphasis in personnel and process management Solid team player with proven ability to establish rapport with clients Skills Type 50 wpm and 10 Key by touch Microsoft programs Word Excel Access Outlook PowerPoint Personnel management and Telephone inquiries specialist Excellent customer service skills with call quality and customer satisfaction as primary focus Inbound and Outbound contact center experience Excellent oral and written presentation and seasoned in conflict resolution Experience Hr Specialist May 2007 to Oct 2008 Company Name City State Accenture Peopleline Email Management Team Received inbound emails from clients as initial point of contact to answer questions related to HR policies practices or to resolve issues Helped the client by answering their inquiry resolving their issue or rout

In [86]:
with open(r"C:\Users\surya\Desktop\guvi\NLP\clf.pkl", 'rb') as file:
    pickle.load(file) 
cleaned_resume = NewResume(myresume)
input_features = tfidf.transform([cleaned_resume])
prediction_id = clf.predict(input_features)[0]
category_mapping = {
    19: "HR",
    13: "DESIGNER",
    20: "INFORMATION-TECHNOLOGY",
    23: "TEACHER",
    1: "ADVOCATE",
    9: "BUSINESS-DEVELOPMENT",
    18: "HEALTHCARE",
    17: "FITNESS",
    2: "AGRICULTURE",
    8: "BPO",
    22: "SALES",
    12: "CONSULTANT",
    14: "DIGITAL-MEDIA",
    5: "AUTOMOBILE",
    10: "CHEF",
    16: "FINANCE",
    3: "APPAREL",
    15: "ENGINEERING",
    0: "ACCOUNTANT",
    11: "CONSTRUCTION",
    21: "PUBLIC-RELATIONS",
    7: "BANKING",
    4: "ARTS",
    6: "AVIATION"
}

category_name = category_mapping.get(prediction_id, "Unknown")

print("Predicted Category:", category_name)
print(prediction_id)

Predicted Category: HR
19


In [91]:
myresume = df['Resume_str'][129]

In [92]:
myresume

' INDEPENDENT DESIGNER Professional Summary Independent and results driven Design Professional with an extensive background in the design process Proficient in taking initiative along with a desire to achieve client satisfaction for a record of accomplished results Skills Space Planning Workplace Strategy Project Coordination FFE Expertise Problem Resolution ADA Building Codes Written Verbal Communication Product Specification AutoCAD MS Office Suite Salesforce ADA Specification AutoCAD Strategy Budgets Verbal Communication Change management Written COUNCIL Creativity Customer satisfaction Direction Documentation Innovation Interior design Team lead Leadership Materials Meetings MS Office Suite Office Organizational Personnel Presentations Problem Resolution Processes Procurement Programming Project Coordination Real Estate Retail Sales Space Planning Work History INDEPENDENT DESIGNER 042020 to Current Company Name City State Create Site and Fixture Plans utilizing AutoCAD for Walgreen

In [93]:
with open(r"C:\Users\surya\Desktop\guvi\NLP\clf.pkl", 'rb') as file:
    pickle.load(file) 
cleaned_resume = NewResume(myresume)
input_features = tfidf.transform([cleaned_resume])
prediction_id = clf.predict(input_features)[0]
category_mapping = {
    19: "HR",
    13: "DESIGNER",
    20: "INFORMATION-TECHNOLOGY",
    23: "TEACHER",
    1: "ADVOCATE",
    9: "BUSINESS-DEVELOPMENT",
    18: "HEALTHCARE",
    17: "FITNESS",
    2: "AGRICULTURE",
    8: "BPO",
    22: "SALES",
    12: "CONSULTANT",
    14: "DIGITAL-MEDIA",
    5: "AUTOMOBILE",
    10: "CHEF",
    16: "FINANCE",
    3: "APPAREL",
    15: "ENGINEERING",
    0: "ACCOUNTANT",
    11: "CONSTRUCTION",
    21: "PUBLIC-RELATIONS",
    7: "BANKING",
    4: "ARTS",
    6: "AVIATION"
}

category_name = category_mapping.get(prediction_id, "Unknown")

print("Predicted Category:", category_name)
print(prediction_id)

Predicted Category: DESIGNER
13


In [97]:
myresume = df['Resume_str'][2000]

In [98]:
myresume

' ASSISTANT PROJECT MANAGERDESIGN BUILD CONSTRUCTION COORDINATOR Summary Highly motivated Civil Engineer with 16 years of experience working for one of North Americas largest and most respected construction and engineering organizations with revenues of more than 11 billion and consistently ranking among the top five contractors by Engineering News Record My experience with Kiewit has prepared me to successfully plan and execute projects while promoting safety quality schedule and cost performance My career experience ranges from construction design coordinator on a multi billion dollar design build project to on site project engineering and project management for self performing heavy civil construction work ranging from 545 million to 31 million dollars Excellent communicator skilled in construction means and methods cost controls scheduling claims analysis and contract administration As the Field Engineer and Project Controls Manager on one of the largest design build project in Mar

In [99]:
with open(r"C:\Users\surya\Desktop\guvi\NLP\clf.pkl", 'rb') as file:
    pickle.load(file) 
cleaned_resume = NewResume(myresume)
input_features = tfidf.transform([cleaned_resume])
prediction_id = clf.predict(input_features)[0]
category_mapping = {
    19: "HR",
    13: "DESIGNER",
    20: "INFORMATION-TECHNOLOGY",
    23: "TEACHER",
    1: "ADVOCATE",
    9: "BUSINESS-DEVELOPMENT",
    18: "HEALTHCARE",
    17: "FITNESS",
    2: "AGRICULTURE",
    8: "BPO",
    22: "SALES",
    12: "CONSULTANT",
    14: "DIGITAL-MEDIA",
    5: "AUTOMOBILE",
    10: "CHEF",
    16: "FINANCE",
    3: "APPAREL",
    15: "ENGINEERING",
    0: "ACCOUNTANT",
    11: "CONSTRUCTION",
    21: "PUBLIC-RELATIONS",
    7: "BANKING",
    4: "ARTS",
    6: "AVIATION"
}

category_name = category_mapping.get(prediction_id, "Unknown")

print("Predicted Category:", category_name)
print(prediction_id)

Predicted Category: CONSTRUCTION
11
